In [1]:
import time
import socket
from queue import Queue 
from threading import Thread, Condition
from ipywidgets import widgets
from IPython.display import display
import random
from collections import namedtuple

In [2]:
class EXP_Output:
    def __init__(self):
        #self.out = widgets.Output(layout={'border': '1px solid black'})
        #display(self.out)
        #with self.out:
        #print("Experiment outputs")

        self.Queue_Print = Queue()
        self.thread = Thread(target=self.loop_Print)
        self.thread.start()

    def print(self, msg):
        text_out = time.strftime("%H:%M:%S", time.localtime()) + "> "
        self.Queue_Print.put(text_out+msg)

    def loop_Print(self):
        while True:
            msg = self.Queue_Print.get()
            #with self.out:
            print(msg)
            self.Queue_Print.task_done()

In [3]:
class PLE_TR:

    def __init__(self, name, Socket, AP_Tx, AP_Rx, exp_out):
        
        self.Name      = name
        self.Socket    = Socket
        self.AP_Tx     = AP_Tx
        self.AP_Rx     = AP_Rx
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
    def loop_Tx(self):
        
        global thread_running
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
        
            # get a message from queue
            msg = self.Queue_Tx.get()
            
            text_out = self.Name + " Tx: message: " + str(msg)
            self.exp_out.print(text_out)
            
            # sending the message using socket
            msg_bytes = str.encode(msg)
            if (random.random() < 0.15):  # 3.a.1) 15% packet loss probability
                self.exp_out.print("packet loss")
            elif random.random() < 0.05:
                # 3.a.2) duplicating the transmission
                self.exp_out.print("transmission duplication")
                self.Socket.sendto(msg_bytes, self.AP_Tx)
                self.Socket.sendto(msg_bytes, self.AP_Tx)
            else:
                # sending the message using socket
                self.Socket.sendto(msg_bytes, self.AP_Tx)

    def loop_Rx(self):
        
        global thread_running
        global bufferSize
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        # binding the socket with the IP and port
        self.Socket.bind(self.AP_Rx)
        
        while (thread_running == True):
        
            # get a message from socket, this thread is blocked here
            msg_addr = self.Socket.recvfrom(bufferSize)

            # 3.b) the delay should be in range of 0 to 3 seconds
            delay = random.uniform(0, 3)
            time.sleep(delay)
    
            msg  = msg_addr[0].decode('utf-8')
            addr = msg_addr[1]
            
            text_out = self.Name + " Rx: from " + str(addr) + ": " + str(msg)
            self.exp_out.print(text_out)
            
            self.Queue_Rx.put(msg)
            
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def receive(self):
        return self.Queue_Rx.get()

In [4]:
class MLE_TR:

    Message = namedtuple('message', ['exp_id', 'msg_id', 'transmission_time'])
    Message_Received = namedtuple('message_received', ['message', 'time_received'])
    
    def __init__(self, name, lower_TR, exp_out):
        
        self.Lower_TR = lower_TR
        self.Name = name
        self.Exp_ID = 0
        self.exp_out = exp_out
        self.Messages_Received = dict()

        # create a button object
        self.Button_Start =  widgets.Button(
                            description='Start Experiment',
                            disabled=False,
                            button_style='', 
                            tooltip='Click me to start a measurement experiment',
                        )

        self.Button_Show_Exp = widgets.Button(
            description='Show Results',
            disabled=False,
            button_style='',
            tooltip='Click me to show the results of the experiment'
        )
        
        # Define a function to handle the click action on the button
        def on_start_exp_button_clicked(b):

            global thread_running
            
            #msg = self.Text.value
            #self.exp_out.print("ALE_0: received input: "+str(msg))
            prefix = f"{self.Name} Tx"
            self.exp_out.print(f"{prefix}: Sending 100 messages")
                
            if not thread_running:
                return

            self.Exp_ID += 1
            for i in range(100):
                msg = f"{self.Exp_ID}:{i + 1}:{time.time_ns()}"
                self.Lower_TR.send(msg)
                self.exp_out.print(f"{prefix}: Sending {msg}")
        
        # Attach the event to the button
        self.Button_Start.on_click(on_start_exp_button_clicked)
        self.Button_Show_Exp.on_click(self.generate_measurements)
        
        # Display the widget
        display(self.Button_Start, self.Button_Show_Exp)

    def loop_Rx(self):
        
        global thread_running
        
        prefix = f"{self.Name} Rx"
        self.exp_out.print(f"{prefix}: loop_Rx starting")
    
        while thread_running:
            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            received = time.time_ns()
            
            text_out = f"{prefix}: received {msg}"
            self.exp_out.print(text_out)

            # Parse received message
            msg_parsed = msg.split(':')
            msg_exp_id = int(msg_parsed[0].strip())
            msg_id = int(msg_parsed[1].strip())
            msg_time = int(msg_parsed[2].strip())
            
            message = self.Message(msg_exp_id, msg_id, msg_time)
            message_received = self.Message_Received(message, received)

            if msg_exp_id not in self.Messages_Received:
                self.Messages_Received[msg_exp_id] = [message_received]
            else:
                self.Messages_Received[msg_exp_id].append(message_received)

    def generate_measurements(self, b):
        for exp_id, messages in self.Messages_Received.items():
            total_delay = 0
            msg_lost = 0
            unique_msg_received = set()
            msg_duplicated = 0

            prev_msg = 0

            messages.sort(key=lambda m: m.message.msg_id)
            self.exp_out.print(f"Calculating measurements for {len(messages)} messages")
        
            for msg in messages:
                total_delay += msg.time_received - msg.message.transmission_time
            
                # Detect loss rate
                msg_id = msg.message.msg_id
                #if prev_msg + 1 != msg_id:
                #    msg_lost += msg_id - (prev_msg + 1)
                #    prev_msg = msg_id

                # Detect duplication rate
                if msg_id in unique_msg_received:
                    msg_duplicated += 1
                else:
                    unique_msg_received.add(msg_id)

            expected_msgs_received = set(range(1, 101))
            lost_msgs = expected_msgs_received - unique_msg_received
            
            msg_recv_len = len(messages)
            average_delay = total_delay / msg_recv_len
            loss_rate = len(lost_msgs) / 100
            duplication_rate = msg_duplicated / 100

            self.exp_out.print(f"--- Experiment {exp_id} ---")
            self.exp_out.print(f"Average delay: {average_delay:.3f} ns")
            self.exp_out.print(f"Average loss rate: {loss_rate * 100:.2f}%; lost {len(lost_msgs)} messages")
            self.exp_out.print(f"Average duplication rate: {duplication_rate * 100:.2f}%; duplicated {msg_duplicated} messages")


In [5]:
thread_running = False
bufferSize = 1024

# (0) creating an output object
exp_out = EXP_Output()

# (1) create physical layer entities
AP_local_1  = ("127.0.0.1", 30000)
AP_remote_1 = ("127.0.0.1", 31111)
Socket_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_1       = PLE_TR("PLE_Alice", Socket_1, AP_remote_1, AP_local_1, exp_out)

# (2) create date link layer entities
#DLE_1 = DLE_TR_FSM("DLE_Alice", PLE_1, exp_out)

# (3) create application layer entities
#ALE_0 = ALE_TextInput(exp_out)
#ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1, exp_out)
MLE_1 = MLE_TR("MLE_Alice", PLE_1, exp_out)

Button(description='Start Experiment', style=ButtonStyle(), tooltip='Click me to start a measurement experimen…

Button(description='Show Results', style=ButtonStyle(), tooltip='Click me to show the results of the experimen…

18:27:59> PLE_Alice: loop_Tx starting
18:27:59> PLE_Alice: loop_Rx starting
18:27:59> MLE_Alice Rx: loop_Rx starting
18:27:59> PLE_Bob: loop_Tx starting
18:27:59> PLE_Bob: loop_Rx starting
18:27:59> MLE_Bob Rx: loop_Rx starting
18:28:00> MLE_Alice Tx: Sending 100 messages
18:28:00> MLE_Alice Tx: Sending 1:1:1760740080223044525
18:28:00> MLE_Alice Tx: Sending 1:2:1760740080223055815
18:28:00> MLE_Alice Tx: Sending 1:3:1760740080223060575
18:28:00> MLE_Alice Tx: Sending 1:4:1760740080223063755
18:28:00> MLE_Alice Tx: Sending 1:5:1760740080223066665
18:28:00> MLE_Alice Tx: Sending 1:6:1760740080223069435
18:28:00> MLE_Alice Tx: Sending 1:7:1760740080223072325
18:28:00> MLE_Alice Tx: Sending 1:8:1760740080223075265
18:28:00> MLE_Alice Tx: Sending 1:9:1760740080223078065
18:28:00> MLE_Alice Tx: Sending 1:10:1760740080223080875
18:28:00> MLE_Alice Tx: Sending 1:11:1760740080223083745
18:28:00> MLE_Alice Tx: Sending 1:12:1760740080223086925
18:28:00> MLE_Alice Tx: Sending 1:13:176074008022308

In [6]:
# (4) create physical layer entities
AP_local_2  = ("127.0.0.1", 31111)
AP_remote_2 = ("127.0.0.1", 30000)
Socket_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2       = PLE_TR("PLE_Bob", Socket_2, AP_remote_2, AP_local_2, exp_out)

# (5) create date link layer entities
#DLE_2 = DLE_TR_FSM("DLE_Bob", PLE_2, exp_out)

# (6) create application layer entities
#ALE_3 = ALE_TextInput(exp_out)
#ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2, exp_out)
MLE_2 = MLE_TR("MLE_Bob", PLE_2, exp_out)

Button(description='Start Experiment', style=ButtonStyle(), tooltip='Click me to start a measurement experimen…

Button(description='Show Results', style=ButtonStyle(), tooltip='Click me to show the results of the experimen…

In [7]:
# start the loops of all entities
# all loops must be blocked at a certain position

#t1_1 = Thread(target = ALE_1.loop_Tx, args = ()) 
#t2_1 = Thread(target = ALE_1.loop_Rx, args = ()) 
#t3_1 = Thread(target = DLE_1.loop_Tx, args = ())
#t4_1 = Thread(target = DLE_1.loop_Rx, args = ())
t5_1 = Thread(target = PLE_1.loop_Tx, args = ()) 
t6_1 = Thread(target = PLE_1.loop_Rx, args = ())
t7_1 = Thread(target = MLE_1.loop_Rx, args = ())
#f1_1 = Thread(target = DLE_1.loop_FSM, args = ())
#f2_1 = Thread(target = DLE_1.loop_timer, args = ())


#t1_2 = Thread(target = ALE_2.loop_Tx, args = ()) 
#t2_2 = Thread(target = ALE_2.loop_Rx, args = ()) 
#t3_2 = Thread(target = DLE_2.loop_Tx, args = ())
#t4_2 = Thread(target = DLE_2.loop_Rx, args = ())
t5_2 = Thread(target = PLE_2.loop_Tx, args = ()) 
t6_2 = Thread(target = PLE_2.loop_Rx, args = ())
t7_2 = Thread(target = MLE_2.loop_Rx, args = ()) 
#f1_2 = Thread(target = DLE_2.loop_FSM, args = ())
#f2_2 = Thread(target = DLE_2.loop_timer, args = ())

thread_running = True

t5_1.start()
t6_1.start()
t7_1.start()

t5_2.start()
t6_2.start()
t7_2.start()
